In [1]:
import tensorflow as tf
import os
import zipfile

DESIRED_ACC = 0.999

In [2]:
!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

--2021-01-08 15:26:01--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 34.64.4.80, 34.64.4.16
Connecting to storage.googleapis.com (storage.googleapis.com)|34.64.4.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  10.3MB/s    in 0.2s    

2021-01-08 15:26:02 (10.3 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [3]:
zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

In [4]:
class CB(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>DESIRED_ACC):
            print("\nCB")
            self.model.stop_training = True
            
callbacks = CB()

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1/255.0)

train_generator = train_datagen.flow_from_directory(
        "/tmp/h-or-s",
        target_size = (150, 150),
        batch_size = 10,
        class_mode = 'binary'
)


Found 80 images belonging to 2 classes.


In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape = (150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [7]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer = RMSprop(lr=0.001), loss = 'binary_crossentropy', metrics=['accuracy'])

In [8]:
history = model.fit(
    train_generator,
    steps_per_epoch = 8,
    epochs = 15,
    verbose = 1,
    callbacks = [callbacks]
)

Epoch 1/15
8/8 [==============================] - 1s 101ms/step - loss: 3.0937 - accuracy: 0.5375
Epoch 2/15
8/8 [==============================] - 1s 100ms/step - loss: 0.4206 - accuracy: 0.7750
Epoch 3/15
8/8 [==============================] - 1s 96ms/step - loss: 0.4337 - accuracy: 0.7875
Epoch 4/15
8/8 [==============================] - 1s 104ms/step - loss: 0.2391 - accuracy: 0.8500
Epoch 5/15
8/8 [==============================] - 1s 104ms/step - loss: 0.1713 - accuracy: 0.9500
Epoch 6/15
8/8 [==============================] - 1s 103ms/step - loss: 0.1601 - accuracy: 0.9250
Epoch 7/15
8/8 [==============================] - 1s 99ms/step - loss: 0.1171 - accuracy: 0.9625
Epoch 8/15
8/8 [==============================] - ETA: 0s - loss: 0.0457 - accuracy: 1.0000
CB
8/8 [==============================] - 1s 103ms/step - loss: 0.0457 - accuracy: 1.0000
